# Exploratory Data Analysis

*Justin R. Garrard*

### Setup

In [2]:
# Import libraries
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from ipywidgets import * 

In [3]:
# Declare global variables
DATA_DIR = os.path.join('../data/processed')
DATA_FILE = os.path.join(DATA_DIR, 'processed.csv')
plt.style.use('ggplot')

In [5]:
# Useful functions
def null_counter(df):
    record_nulls = []
    for col in df.columns:
        nulls = df[col].isnull().sum()
        percent_null = round((nulls / df.shape[0]) * 100, 2)
        record_nulls.append([col, nulls, percent_null])
    output = pd.DataFrame(record_nulls, columns=['Attribute', 'Null Count', '% Null'])
    return output

def get_year_range(df):
    year_range = list(df['year'].unique())
    year_range.sort()
    return year_range

### Preliminaries

In this section we preview the data, taking note of its scope and completeness.

***High-Level Overview***

* We have ~585,000 records with 51 indicators

* The data ranges from 1986 to 2018

In [18]:
# Load and preview data
edu_df = pd.read_csv(DATA_FILE)

nRow, nCol = edu_df.shape
print(f'There are {nRow} rows and {nCol} columns.')
print('')

YEAR_RANGE = get_year_range(edu_df)
print(f'Data spans the years {YEAR_RANGE[0]} to {YEAR_RANGE[-1]}.')
print('')

print('Available columns include:')
display(null_counter(edu_df))

There are 585650 rows and 51 columns.

Data spans the years 1986 to 2018.

Available columns include:


,Attribute,Null Count,% Null
0,leaid,0,0.00
1,year,0,0.00
2,read_test_num_valid,441477,75.38
3,read_test_pct_prof_midpt,441477,75.38
4,math_test_num_valid,441567,75.40
5,math_test_pct_prof_midpt,441567,75.40
6,lea_name,2521,0.43
7,state_leaid,2565,0.44
8,street_location,203079,34.68
9,city_location,202967,34.66


In [6]:
# Partition columns for easier analysis
location_cols = ["leaid", "year", "lea_name", "state_leaid", 
                 "street_location", "city_location", "state_location",
                 "zip_location", "zip4_location", "fips", "agency_type",
                 "number_of_schools", "county_code", "county_name", 
                 "latitude", "longitude", "cbsa", "cbsa_type", "csa"]

demographic_cols = ["leaid", "year", "teachers_total_fte", "staff_total_fte",
                    "spec_ed_students", "english_language_learners", 
                    "enrollment_x", "enrollment_y", "cmsa", "district_id",
                    "est_population_total", "est_population_5_17", 
                    "est_population_5_17_poverty", "est_population_5_17_poverty_pct",
                    "est_population_5_17_pct", "enrollment_fall_responsible", 
                    "enrollment_fall_school"]

assessment_cols = ["leaid","year","read_test_num_valid", 
                    "read_test_pct_prof_midpt", "math_test_num_valid",
                    "math_test_pct_prof_midpt", "grad_rate_midpt"]

finance_cols = ["leaid", "year", "rev_total", "rev_fed_total", "rev_state_total",
                "rev_local_total", "exp_total", "exp_current_instruction_total",
                "exp_current_supp_serve_total", "exp_current_other", "exp_nonelsec",
                "salaries_total", "benefits_employee_total", "debt_longterm_outstand_beg_FY"]

## Sanity check, should be empty if all cols are accounted for
subsets = set(location_cols + demographic_cols + assessment_cols + finance_cols)
print(set(edu_df.columns) - subsets)

set()


### Location Data

**High-Level Overview**

* Significant portions of the dataset are only available for certain year ranges (i.e. lat/long records begin in 2006)

* The vast majority (75%) of school districts have between 1-5 schools. However, at least two school districts have upwards of 1000 schools.


In [7]:
# High-Level Overview
location_df = edu_df[location_cols]
display(null_counter(location_df))
print('')
display(location_df.head())

,Attribute,Null Count,% Null
0,leaid,0,0.00
1,year,0,0.00
2,lea_name,2521,0.43
3,state_leaid,2565,0.44
4,street_location,203079,34.68
5,city_location,202967,34.66
6,state_location,202963,34.66
7,zip_location,202967,34.66
8,zip4_location,284276,48.54
9,fips,2504,0.43


,leaid,year,lea_name,state_leaid,street_location,city_location,state_location,zip_location,zip4_location,fips,agency_type,number_of_schools,county_code,county_name,latitude,longitude,cbsa,cbsa_type,csa
0,100005,2013,ALBERTVILLE CITY,101,107 WEST MAIN ST,ALBERTVILLE,AL,35950.0,25,1.0,1.0,6.0,1095.0,MARSHALL COUNTY,34.267502,-86.208603,10700.0,2.0,290.0
1,100006,2013,MARSHALL COUNTY,048,12380 US HWY 431 SOUTH,GUNTERSVILLE,AL,35976.0,9351,1.0,1.0,16.0,1095.0,MARSHALL COUNTY,34.305000,-86.286697,10700.0,2.0,290.0
2,100007,2013,HOOVER CITY,158,2810 METROPOLITAN WAY,HOOVER,AL,35243.0,5500,1.0,1.0,17.0,1073.0,JEFFERSON COUNTY,33.406200,-86.766899,13820.0,1.0,142.0
3,100008,2013,MADISON CITY,169,211 CELTIC DR,MADISON,AL,35758.0,1615,1.0,1.0,11.0,1089.0,MADISON COUNTY,34.687302,-86.744904,26620.0,1.0,290.0
4,100011,2013,LEEDS CITY,167,8121 PARKWAY DR,LEEDS,AL,35094.0,NaN,1.0,1.0,3.0,1073.0,JEFFERSON COUNTY,33.543301,-86.541298,13820.0,1.0,142.0


In [8]:
# General statistics
display(location_df.describe())

,leaid,year,zip_location,fips,agency_type,number_of_schools,county_code,latitude,longitude,cbsa,cbsa_type,csa
count,5.856500e+05,585650.000000,382683.000000,583146.000000,583129.000000,579245.000000,583029.000000,242140.000000,242140.000000,314631.000000,314631.000000,314629.000000
mean,2.984780e+06,2002.479623,51242.348071,29.711683,1.870800,5.415857,29610.561507,39.642938,-91.786224,24041.530854,0.435396,138.533174
std,1.464476e+06,9.560158,28943.066672,14.638911,1.823968,19.461834,14586.647008,4.665949,15.199142,17436.385912,1.454066,183.776201
min,7.230000e+03,1986.000000,-2.000000,1.000000,1.000000,-2.000000,-2.000000,-14.278038,-170.695602,-2.000000,-2.000000,-2.000000
25%,1.810080e+06,1994.000000,27105.000000,18.000000,1.000000,1.000000,18109.000000,36.314642,-98.994393,11020.000000,1.000000,-2.000000
50%,3.025020e+06,2003.000000,55388.000000,30.000000,1.000000,3.000000,30089.000000,40.448490,-89.554304,24940.000000,1.000000,34.000000
75%,4.023190e+06,2011.000000,74601.000000,40.000000,2.000000,5.000000,40095.000000,42.650081,-80.398127,38060.000000,1.000000,260.000000
max,7.800030e+06,2018.000000,99929.000000,78.000000,9.000000,1756.000000,78030.000000,71.299927,145.755997,79600.000000,2.000000,950.000000


In [24]:
# Interactive plot
# Metrics by state, by year
%matplotlib notebook

year_range = get_year_range(location_df)
metrics = list(location_df.columns)[2:]

@interact(year=(year_range[0],year_range[-1],1), metric=metrics)
def metric_explorer(year, metric):
    # Clear any old figures
    plt.close()
    
    # Take a snapshot of the data for the given year
    snapshot = location_df[location_df['year'] == year].copy()
    snapshot.sort_values(metric, ascending=True, inplace=True)
    y_pos = np.arange(len(snapshot[metric]))
    
    # Make a plot to match states to the chosen metric
    plt.figure(figsize=(8, 8), num='Location Metric Explorer Tool')
    plt.scatter(snapshot['leaid'], snapshot[metric], alpha=0.5)
    plt.xlabel('leaid')
    plt.ylabel(metric)
    plt.title(f'{metric}: {year}')

    
interactive_plot = interactive(metric_explorer,
                               year=2005,
                               metric=metrics[0],
                               sort_by=sort_by_keys_mappings[0])

interactive(children=(IntSlider(value=2002, description='year', max=2018, min=1986), Dropdown(description='met…

### Demographic Data

This section describes data taken primarily from the Directory and Saipe sets, as well some additional enrollment information from the Finance set.

### Achievement Data

This section describes data taken from the Assessment and Grad Rate sets.

### Financial Data

This section describes data taken from the Financial set.